In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [47]:
spotify = pd.read_csv('music_genre.csv')

# Data cleaning

Como o instance_id identifica a música de forma única, verificamos se há valores duplicados para essa feature.

In [48]:
spotify[spotify['instance_id'].duplicated()]

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A saída anterior mostra que os valores de instance_id duplicados são, na verdade, nulos. Então fazemos a remoção das linhas onde o instance_id é nulo.

In [49]:
spotify = spotify[~spotify['instance_id'].isna()]

É importante entender quais features podem ajudar na predição da feature de interesse (music_genre). As features instance_id, artist_name e track_name devem ser retiradas, pois é uma informação única para aquela música, ou seja, não é uma informação que podemos generalizar para outras músicas. Além disso, retiramos a feature obtained_date. Não encontramos nenhuma informação dessa feture, mas suspeitamos que refere-se a data que a música foi coletada.

In [50]:
spotify = spotify.drop(['instance_id','artist_name','track_name','obtained_date'], axis = 1)

Agora verificamos o tipo das variáveis. Isso é importante, pois devemos garantir que as features numéricas estejam como numéricas as features categóricas como categóricas.

In [51]:
spotify.dtypes

popularity          float64
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo                object
valence             float64
music_genre          object
dtype: object

A feature tempo é numérica. Precisamos entender porque ela veio como tipo object.

In [54]:
spotify['tempo'].sort_values()

28375    100.0
46597    100.0
38389    100.0
46815    100.0
14217    100.0
         ...  
43350        ?
10361        ?
42625        ?
17409        ?
46955        ?
Name: tempo, Length: 50000, dtype: object

Na saída anterior há alguns valores com '?', indicando que não há informação de tempo para a respectiva música. Logo, tratamos '?' como valor faltante.

In [63]:
spotify['tempo'] = spotify['tempo'].replace('?', np.NaN)

In [64]:
spotify[['key','mode','music_genre']] = spotify[['key','mode','music_genre']].astype('category')
spotify[['tempo']] = spotify[['tempo']].astype('float')

spotify.dtypes

popularity           float64
acousticness         float64
danceability         float64
duration_ms          float64
energy               float64
instrumentalness     float64
key                 category
liveness             float64
loudness             float64
mode                category
speechiness          float64
tempo                float64
valence              float64
music_genre         category
dtype: object

Tudo certo com o tipo das features! 

Nas explicações das features no github, temos os possíveis valores para todas as features. O próximo passo é verificar se há algumas observações com os valores fora do intervalo permitido pelas features.

In [66]:
spotify.describe()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,45020.000000,50000.000000
mean,44.220420,0.306383,0.558241,2.212526e+05,0.599755,0.181601,0.193896,-9.133761,0.093586,119.952961,0.456264
std,15.542008,0.341340,0.178632,1.286720e+05,0.264559,0.325409,0.161637,6.162990,0.101373,30.641701,0.247119
min,0.000000,0.000000,0.059600,-1.000000e+00,0.000792,0.000000,0.009670,-47.046000,0.022300,34.347000,0.000000
25%,34.000000,0.020000,0.442000,1.748000e+05,0.433000,0.000000,0.096900,-10.860000,0.036100,94.941500,0.257000
50%,45.000000,0.144000,0.568000,2.192810e+05,0.643000,0.000158,0.126000,-7.276500,0.048900,119.879000,0.448000
75%,56.000000,0.552000,0.687000,2.686122e+05,0.815000,0.155000,0.244000,-5.173000,0.098525,140.471250,0.648000
max,99.000000,0.996000,0.986000,4.830606e+06,0.999000,0.996000,1.000000,3.744000,0.942000,220.276000,0.992000


A saída acima mostra algumas estatísticas sumárias para as features numéricas. A única feature que identificamos inconsistência é a duration_ms, pois há valores negativos e isso não faz sentido para uma feature que é, essencialmente, positiva. Para uma análise mais elaborada, mostramos na saída abaixo um resumo dos valores de duration_ms menor ou igual a zero.

In [73]:
spotify[spotify['duration_ms'] <= 0]['duration_ms'].describe()

count    4939.0
mean       -1.0
std         0.0
min        -1.0
25%        -1.0
50%        -1.0
75%        -1.0
max        -1.0
Name: duration_ms, dtype: float64

A saída anterior mostra que os valores de duration_ms que não são positivos recebem o valor -1, pois o mínimo é igual ao máximo. Isso sugere que as músicas que não tiveram a duração_ms armazenada receberam o valor -1. Logo, tratamos -1 como valor faltante.

In [74]:
spotify['duration_ms'] = spotify['duration_ms'].replace(-1, np.NaN)

In [65]:
spotify.describe(include = 'category')

,key,mode,music_genre
count,50000,50000,50000
unique,12,2,10
top,G,Major,Alternative
freq,5727,32099,5000


A saída acima mostra algumas estatísticas sumárias para as features categóricas. A partir dela não identificamos nenhuma inconsistência com as features numéricas.

Agora precisamos identificar quais features tem valores faltantes.

In [78]:
spotify.isna().any()

popularity          False
acousticness        False
danceability        False
duration_ms          True
energy              False
instrumentalness    False
key                 False
liveness            False
loudness            False
mode                False
speechiness         False
tempo                True
valence             False
music_genre         False
dtype: bool

In [86]:
spotify.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 50004
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   popularity        50000 non-null  float64 
 1   acousticness      50000 non-null  float64 
 2   danceability      50000 non-null  float64 
 3   duration_ms       45061 non-null  float64 
 4   energy            50000 non-null  float64 
 5   instrumentalness  50000 non-null  float64 
 6   key               50000 non-null  category
 7   liveness          50000 non-null  float64 
 8   loudness          50000 non-null  float64 
 9   mode              50000 non-null  category
 10  speechiness       50000 non-null  float64 
 11  tempo             45020 non-null  float64 
 12  valence           50000 non-null  float64 
 13  music_genre       50000 non-null  category
dtypes: category(3), float64(11)
memory usage: 4.7 MB


As features com valores faltantes são: duration_ms e tempo. Como, aproximadamente, 10% são valores faltantes para as duas features, inputamos os valores faltantes com a média.

In [92]:
spotify = spotify.fillna(spotify.mean()[['duration_ms','tempo']])

Com as manipulações que fizemos até aqui, podemos partir para a análise exploratória.

# Análise exploratória

Antes de começar a análise vamos definir algumas nomenclaturas para facilitar o entendimento: a feature que queremos prever, ou classificar, chamamos de **variável resposta**, ou seja, music_genre é a variável resposta; as outras features chamamos de **preditoras**, pois são usadas para predizer a variável resposta.

### Análise das preditoras categóricas versus variável resposta

### Análise das preditoras numéricas versus variável resposta